## SDKを用いた計装

必要なライブラリをダウンロードします。

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


実行に必要な環境変数を読み込みます。

In [8]:
import os, warnings, time
from dotenv import load_dotenv

load_dotenv("../.env")

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://langfuse-server:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

LangfuseのSDKクライアントを初期化します。

In [9]:
from langfuse import Langfuse

langfuse = Langfuse(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

### Low-Level SDK

Low-Level SDKを用いてTraceと各種Observation(Event/Span/Generation)の確認をします。

In [7]:
# Traceを作成する
trace = langfuse.trace(
    name="Ask the BigBaBy"
)

# Observation - EventをTraceに追加する
event = trace.event(
    name="Meet the BigBaBy"
)

# Observation - SpanをTraceに追加する
span = trace.span(
    name="Search the BigBaBy's dictionary"
)
# 2秒間スリープ（同期コールする検索システムを想定）
time.sleep(2)
span.end()

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    llm = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)

input = [
    {"role": "user", "content": "おすすめの肉料理おしえて"}
]

generation = trace.generation(
    name="Ask the BigBaBy",
    model=model,
    input=input
)

result = llm.invoke(input=input)


### Decorator

`@obserbe()`デコレーターを用いてTraceと各種Observation(Span/Generation)の確認をします。

In [19]:
from langfuse.decorators import observe, langfuse_context

@observe()
def meet_the_bigbaby():
    print("Meet the BigBaBy!")

@observe(name="Search the BigBaBy's dictionary")
def search_the_bigbaby_dict():
    time.sleep(2)

@observe(name="Ask the BigBaBy", as_type="generation")
def ask_the_bigbaby(query: str):
    if os.getenv("COHERE_API_KEY") == None:
        from langchain_openai.chat_models import ChatOpenAI
        openai_api_key = os.getenv("OPENAI_API_KEY")
        model = "gpt-4o-mini"
        llm = ChatOpenAI(api_key=openai_api_key, model=model)
    else:
        from langchain_cohere.chat_models import ChatCohere
        cohere_api_key = os.getenv("COHERE_API_KEY")
        model = "command-r-plus"
        llm = ChatCohere(cohere_api_key=cohere_api_key, model=model)
    input = [
        {"role": "user", "content": query}
    ]
    result = llm.invoke(input=input)
    return result.content

@observe(name="Ask the BigBaby")
def main():
    meet_the_bigbaby()
    search_the_bigbaby_dict()
    return ask_the_bigbaby(query="肉料理のおすすめ教えて")

langfuse_context.configure(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)
response = main()
print(f"{response=}")

Meet the BigBaBy!
response='喜んでいただけるか分かりませんが、いくつかおすすめの肉料理を紹介します。\n\n- ローストビーフ： 牛肉の塊をオーブンでゆっくりと焼き上げた料理です。ジューシーでやわらかな食感と、深い味わいが魅力です。\n- 豚の角煮： 豚バラ肉を醤油や砂糖などで長時間煮込んだ料理です。とろけるような食感と、甘辛い味付けがご飯にもお酒にもよく合います。\n- チキンティッカ： インド料理のグリルチキンです。ヨーグルトや香辛料に漬け込んだ鶏肉を串に刺して焼き上げます。香り高く、ジューシーな味わいが楽しめます。\n- ラムチョップ： ラム肉のリブ（あばら骨）部分をグリルしたり、オーブンで焼いたりした料理です。やわらかな食感と、独特の香りが魅力です。\n- 牛タンのステーキ： 牛タンを厚切りにしてステーキにした料理です。やわらかな食感と、深い味わいが楽しめます。\n\n以上、お好みの料理はありましたか？'
